In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm
import re
import time
from functools import partial
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import selenium.webdriver as webdriver
from selenium.webdriver.support.wait import WebDriverWait

In [2]:
main_url='https://www.saha.go.kr/portal/contents.do?mId=0201010000'
stop_date='2020-12-01'

In [3]:
# 각각의 리스트 생성
date_list=[]
title_list=[]
text_list=[] 

In [4]:
def clean_text(text):
    """
    한글,영문,숫자 남기고 제거한다.
    """
    
    text = re.sub(r'[^ ㄱ-ㅣ가-힣A-Za-z]', '', text) #특수기호 제거, 정규 표현식    

    return text

In [5]:
options=webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('disable-gpu')



driver = webdriver.Chrome('./chromedriver',options=options)
#driver2 = webdriver.Chrome('./chromedriver',options=options)


driver.implicitly_wait(10)
#driver2.implicitly_wait(10)

In [6]:
driver.get(main_url)

In [7]:
driver.find_element_by_xpath('//*[@id="conts"]/div/a').click()

In [8]:
# 최근 열린 탭으로 전환
driver.switch_to.window(driver.window_handles[-1])

# 로딩 기다리기
time.sleep(5)

In [9]:
next_b=driver.find_element_by_xpath('//*[@id="navigator"]/a[12]/i')

In [10]:
def get_complain():
    time.sleep(0.2)
    list_com=driver.find_elements_by_css_selector('#dataSetTb > table > tbody > tr > td.td-list > a')
    for x in tqdm(range(len(list_com))):
        list_com=driver.find_elements_by_css_selector('#dataSetTb > table > tbody > tr > td.td-list > a')
        list_com[x].click()
        time.sleep(0.2)
        title=driver.find_element_by_css_selector('body > main > div > fieldset > table > tbody > tr:nth-child(1) > td:nth-child(4)').text
        text=driver.find_element_by_css_selector('body > main > div > fieldset > table > tbody > tr:nth-child(4) > td').text
        date=driver.find_element_by_css_selector('body > main > div > fieldset > table > tbody > tr:nth-child(2) > td:nth-child(4)').text
        date=date.split(' ')[0]
        
        if date==stop_date:
            return
        title=clean_text(title)
        text=clean_text(text)
        title_list.append(title)
        text_list.append(text)
        date_list.append(date)
        len_list=len(driver.find_elements_by_class_name('content-explain'))
        list_b=driver.find_element_by_css_selector('body > main > div > div.btn-group.align-right > a')
            
        try:
            list_b.click()
        except:
            pass
        time.sleep(0.2)
    next_b=driver.find_element_by_xpath('//*[@id="navigator"]/a[12]/i')
    next_b.click()
    get_complain()
    
    

In [ ]:
get_complain()

In [ ]:
driver.close()
driver.close()



In [ ]:
import pandas as pd

In [ ]:
df=pd.DataFrame({"date":date_list,"title":title_list,"text":text_list}) 

In [ ]:
df